In [3]:
import sys
sys.path.append('..')
sys.path.append('/Users/jh7685/Documents/Projects/pysurfer')
import numpy as np
from pysurfer.mgz_helper import map_values_as_mgz
import os
from sfp_nsdsyn import *
import warnings
from itertools import product
warnings.filterwarnings("ignore", category=UserWarning)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
nsd_dir ='/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset'
output_dir='/Volmes/server/Projects/sfp_nsd/derivatives'
#freesurfer_dir='/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset/nsddata/freesurfer'

In [5]:
subj='subj01'
hemi='lh'
stim_list = ['annulus', 'pinwheel', 'forward spiral', 'reverse spiral']
betas = os.path.join(nsd_dir, 'nsddata_betas', 'ppdata', f'{subj}', 'nativesurface', 'nsdsyntheticbetas_fithrf_GLMdenoise_RR', f'{hemi}.betas_nsdsynthetic.hdf5')
design_mat=os.path.join(nsd_dir, 'nsddata', 'experiments', 'nsdsynthetic', 'nsdsynthetic_expdesign.mat')
task_keys=['fixation_task','memory_task']
stim_info=os.path.join(nsd_dir, 'nsdsyn_stim_description.csv')
task_average=True
eccentricity_path= os.path.join(nsd_dir, 'nsddata', 'freesurfer', f'{subj}', 'label',  f'{hemi}.prfeccentricity.mgz')

# Load whole brain betas

In [6]:
betas_df = sfm.get_whole_brain_betas(betas_path=betas, design_mat_path=design_mat,
                          stim_info_path=stim_info,
                          task_keys=task_keys, task_average=False, eccentricity_path=eccentricity_path,
                          x_axis='voxel', y_axis='stim_idx', long_format=True)
betas_df = betas_df.query('names in @stim_list')

# Normalize betas by the frequency magnitude (?)

In [7]:
betas_df['normed_betas'] = bts.normalize_betas_by_frequency_magnitude(betas_df, betas='betas', freq_lvl='freq_lvl')

## NaN values???

In [107]:
tmp_df = betas_df.query('194587 < voxel < 194590')

bts.normalize_betas_by_frequency_magnitude(tmp_df, betas='betas', freq_lvl='freq_lvl')

0      0.363028
1      2.243133
2      0.791020
3      2.831621
4      0.401242
         ...   
379   -2.525187
380    0.310012
381    1.189318
382    0.000000
383    1.048404
Length: 384, dtype: float64

In [135]:
betas_df['normed_betas'] = bts.normalize_betas_by_frequency_magnitude(betas_df, betas='betas', freq_lvl='freq_lvl')
betas_df.query('voxel == 194589')

,stim_idx,image_idx,names,w_r,w_a,phase,class_idx,freq_lvl,voxel,task,betas,eccentricity,local_sf,normed_betas
43587936,0,104,pinwheel,0.0,6.0,0.000000,0,0,194589,fixation,0.353333,2.336213,6.000000,-0.071187
43587937,0,104,pinwheel,0.0,6.0,0.000000,0,0,194589,memory,2.076667,2.336213,6.000000,0.468191
43587938,1,105,pinwheel,0.0,6.0,1.570796,0,0,194589,fixation,1.293333,2.336213,6.000000,-0.102673
43587939,1,105,pinwheel,0.0,6.0,1.570796,0,0,194589,memory,3.410000,2.336213,6.000000,0.898051
43587940,2,106,pinwheel,0.0,6.0,3.141593,0,0,194589,fixation,0.130000,2.336213,6.000000,-0.267636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43588123,93,197,reverse spiral,91.0,-91.0,1.570796,23,5,194589,memory,-1.493333,2.336213,128.693434,NaN
43588124,94,198,reverse spiral,91.0,-91.0,3.141593,23,5,194589,fixation,0.183333,2.336213,128.693434,NaN
43588125,94,198,reverse spiral,91.0,-91.0,3.141593,23,5,194589,memory,0.703333,2.336213,128.693434,NaN
43588126,95,199,reverse spiral,91.0,-91.0,4.712389,23,5,194589,fixation,0.000000,2.336213,128.693434,NaN


In [168]:
new_df = betas_df.query('voxel == 194589')
a = bts.normalize_betas_by_frequency_magnitude(new_df, betas='betas', freq_lvl='freq_lvl')

new_df_2 = pd.merge(new_df[['voxel','betas']], a, on='voxel')
new_df_2['normed_betas'] = np.divide(new_df_2['betas'], new_df_2['length'])
new_df_2

,voxel,betas,length,normed_betas
0,194589,0.353333,0.591375,0.597477
1,194589,2.076667,0.591375,3.511589
2,194589,1.293333,0.591375,2.186993
3,194589,3.410000,0.591375,5.766221
4,194589,0.130000,0.591375,0.219827
...,...,...,...,...
187,194589,-1.493333,0.591375,-2.525187
188,194589,0.183333,0.591375,0.310012
189,194589,0.703333,0.591375,1.189318
190,194589,0.000000,0.591375,0.000000


In [170]:
a = bts.normalize_betas_by_frequency_magnitude(betas_df, betas='betas', freq_lvl='freq_lvl')
a.isna().any()

stim_idx        False
image_idx       False
names           False
w_r             False
w_a             False
phase           False
class_idx       False
freq_lvl        False
voxel           False
task            False
betas           False
eccentricity    False
local_sf        False
normed_betas    False
length          False
dtype: bool

In [161]:
new_df.dtypes

stim_idx         object
image_idx         int64
names            object
w_r             float64
w_a             float64
phase           float64
class_idx         int64
freq_lvl          int64
voxel             int64
task             object
betas           float32
eccentricity    float64
local_sf        float64
normed_betas    float64
length          float64
dtype: object

In [151]:
a.dtypes

voxel       int64
betas     float32
length    float64
normed    float64
dtype: object

In [109]:
new_df[new_df['normed_betas'].isna()][['voxel','betas','normed_betas']]

,voxel,betas,normed_betas
43588032,194589,-0.626667,NaN
43588033,194589,1.370000,NaN
43588034,194589,1.053333,NaN
43588035,194589,0.240000,NaN
43588036,194589,1.186667,NaN
...,...,...,...
50852667,227020,2.403333,NaN
50852668,227020,-8.046667,NaN
50852669,227020,1.286667,NaN
50852670,227020,0.453333,NaN


In [117]:
tmp_df = betas_df.query('194587 < voxel < 194590')
tmp = tmp_df.groupby(['voxel', 'freq_lvl'])['betas'].mean().reset_index()
tmp = tmp.pivot('voxel', 'freq_lvl', 'betas')
length = np.linalg.norm(tmp, axis=1, keepdims=True)
length = np.concatenate((tmp.index.to_numpy().reshape(-1,1), length), axis=1)

In [118]:
length_df = pd.DataFrame(length, columns=['voxel', 'length'])
length_df

,voxel,length
0,194588.0,0.872292
1,194589.0,0.591375


In [119]:
tmp_df = pd.merge(tmp_df[['voxel', 'betas']], length_df, on='voxel')
tmp_df

,voxel,betas,length
0,194588,0.316667,0.872292
1,194588,1.956667,0.872292
2,194588,0.690000,0.872292
3,194588,2.470000,0.872292
4,194588,0.350000,0.872292
...,...,...,...
379,194589,-1.493333,0.591375
380,194589,0.183333,0.591375
381,194589,0.703333,0.591375
382,194589,0.000000,0.591375


In [123]:
tmp_df['normed_betas'] = tmp_df['betas'] / tmp_df['length']
tmp_df.isna()

,voxel,betas,length,normed_betas
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
379,False,False,False,False
380,False,False,False,False
381,False,False,False,False
382,False,False,False,False


In [94]:
betas_df['normed_betas'] = tmp_df['betas'] / tmp_df['length']

In [ ]:
betas

In [95]:
betas_df[betas_df['normed_betas'].isna()][['voxel','betas','normed_betas']]

,voxel,betas,normed_betas
43588032,194589,-0.626667,NaN
43588033,194589,1.370000,NaN
43588034,194589,1.053333,NaN
43588035,194589,0.240000,NaN
43588036,194589,1.186667,NaN
...,...,...,...
50852667,227020,2.403333,NaN
50852668,227020,-8.046667,NaN
50852669,227020,1.286667,NaN
50852670,227020,0.453333,NaN


In [26]:
betas_df[betas_df['normed_betas'].isna()].index.to_list()

[43588032,
 43588033,
 43588034,
 43588035,
 43588036,
 43588037,
 43588038,
 43588039,
 43588040,
 43588041,
 43588042,
 43588043,
 43588044,
 43588045,
 43588046,
 43588047,
 43588048,
 43588049,
 43588050,
 43588051,
 43588052,
 43588053,
 43588054,
 43588055,
 43588056,
 43588057,
 43588058,
 43588059,
 43588060,
 43588061,
 43588062,
 43588063,
 43588064,
 43588065,
 43588066,
 43588067,
 43588068,
 43588069,
 43588070,
 43588071,
 43588072,
 43588073,
 43588074,
 43588075,
 43588076,
 43588077,
 43588078,
 43588079,
 43588080,
 43588081,
 43588082,
 43588083,
 43588084,
 43588085,
 43588086,
 43588087,
 43588088,
 43588089,
 43588090,
 43588091,
 43588092,
 43588093,
 43588094,
 43588095,
 43588096,
 43588097,
 43588098,
 43588099,
 43588100,
 43588101,
 43588102,
 43588103,
 43588104,
 43588105,
 43588106,
 43588107,
 43588108,
 43588109,
 43588110,
 43588111,
 43588112,
 43588113,
 43588114,
 43588115,
 43588116,
 43588117,
 43588118,
 43588119,
 43588120,
 43588121,
 43588122,

In [17]:
betas_df[betas_df['normed_betas'].isna()].index.shape[0]/betas_df.index.shape[0]

0.14285682822293974

# Get sigma_v for each voxel and map it in mgz

In [11]:
sigma_v = bts.get_sigma_v_for_whole_brain(betas_df, betas='normed_betas')

In [12]:
map_values_as_mgz(eccentricity_path, 1/sigma_v,         save_path=f'/Volumes/server/Projects/sfp_nsd/derivatives/sfp_maps/mgzs/nsdsyn/{hemi}.sub-{subj}_value-precision.mgz')

# Plot precision maps

In [22]:
import neuropythy as ny
import ipyvolume as ipv

%gui qt
ny.config['freesurfer_subject_paths'] = '/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset/nsddata/freesurfer'

In [19]:
sub = ny.freesurfer_subject('/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset/nsddata/freesurfer/subj01')
surf = sub.lh.surface('pial')
ny.cortex_plot(surf, color='thickness', cmap='hot', vmin=2, vmax=6)

RuntimeError: load failure: the requested object could not be loaded, probably because you do not have ipyvolume installed correctly